#Setup

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import joblib
import tensorflow as tf

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_check_point = 'bert-base-uncased'

# Initialse base model and tokenizer
base_model = AutoModel.from_pretrained(model_check_point).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_check_point)

# Freeze base model parameters
for param in base_model.parameters():
  param.requires_grad = False

# Read input file
input_file = pd.read_csv('./test.csv', keep_default_na=False) # Change the file name as required

# Classifier class
class MLPClassifier(nn.Module):
  def __init__(self):
      super().__init__()
      self.layers = nn.Sequential(
        nn.Linear(768, 256),
        nn.Dropout(0.3),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.Dropout(0.3),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Linear(128, 1),
        nn.Sigmoid()
      )

  def forward(self, x):
      return self.layers(x)

# Load model
model = joblib.load('./MLP_trained_model.sav').to(device)

# Load test dataset
premises = input_file['premise'].tolist()
hypotheses = input_file['hypothesis'].tolist()

# Tokenise input sequence
encoded_pair = tokenizer(premises, hypotheses, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
token_ids = encoded_pair['input_ids']
attention_masks = encoded_pair['attention_mask']
token_type_ids = encoded_pair['token_type_ids']

# Data loader
test_dataset = TensorDataset(token_ids, attention_masks, token_type_ids)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Prediction

In [ ]:
pre_labels = []

# Set model to evaluation mode
model.eval()
with torch.no_grad():
  for inputs, attention_masks, token_type_ids in test_loader:
    inputs, attention_masks, token_type_ids = inputs.to(device), attention_masks.to(device), token_type_ids.to(device)

    # Forward pass
    outputs = base_model(input_ids=inputs, attention_mask=attention_masks, token_type_ids=token_type_ids) # Generate word embeddings
    pooled_output = outputs[0].mean(dim=1)  # Average pooling the last hidden states generated by base model
    logits = model(pooled_output)
    pre = torch.round(logits).int().data.cpu()  # 1: entailment; 0: non-entailment
    pre_labels.append(pre)

In [ ]:
# Convert list of tensor arrays to list of int
pre_labels = tf.concat(pre_labels, axis=0)
pre_labels = tf.reshape(pre_labels, [-1])
pre_labels = pre_labels.numpy().tolist()

#Output Predictions File

In [ ]:
df = pd.DataFrame({'prediction': pre_labels})
df.to_csv('Group_15_B.csv', header=True, index=False, sep=',')